In [151]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
import time

In [164]:
#initialize chrome driver
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome',**executable_path,headless = True)

#Create empty dictionary to store all data in
mars_dict = {}

In [165]:
#Url of page to be scraped
url = "https://mars.nasa.gov/news/"
browser.visit(url)
browser.links.find_by_partial_text('Next')

html = browser.html
# Create a Beautiful Soup object
soup = bs(html,'html.parser')

In [166]:
#collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
news_title = soup.find_all('li',class_='slide')[0].find(class_='content_title').text
news_p = soup.find_all('li',class_='slide')[0].text

print(news_title)
print('\n')
print(news_p)

mars_dict['news_title'] = news_title
mars_dict['news_p'] = news_p

NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars


In a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. NASA's Perseverance Rover Will Carry First Spacesuit Materials to MarsJuly 28, 2020NASA's Perseverance Rover Will Carry First Spacesuit Materials to MarsIn a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. 


In [168]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to 
#a variable called `featured_image_url`.

pic_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(pic_url)

#click on the 'full image' button to view featured image
browser.find_by_id('full_image').click()
time.sleep(4)
pic = browser.html

# Create a Beautiful Soup object
soup = bs(pic,'html.parser')

featured_image_url = "https://www.jpl.nasa.gov" + soup.find_all('img',class_='fancybox-image')[0]['src']

print(featured_image_url)
mars_dict['featured_image_url'] = featured_image_url

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14924_ip.jpg


In [177]:
#Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page. 
#Save the tweet text for the weather report as a variable called `mars_weather`.

twit_url= "https://twitter.com/marswxreport?lang=en"
browser.visit(twit_url)

browser.links.find_by_partial_text('Next')

twit = browser.html

soup_twit = bs(twit,'html.parser')

weather = soup_twit.find('article',class_="css-1dbjc4n r-1loqt21 r-18u37iz r-1ny4l3l r-o7ynqc r-6416eg").text
mars_weather = weather.strip().replace('Mars Weather@MarsWxReport·9hInSight ','')
print(mars_weather)

mars_dict['mars_weather'] = mars_weather

sol 594 (2020-07-28) low -91.8ºC (-133.2ºF) high -16.6ºC (2.2ºF)
winds from the WNW at 8.0 m/s (17.8 mph) gusting to 20.7 m/s (46.4 mph)
pressure at 7.90 hPa1322


In [170]:
# Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet 
#including Diameter, Mass, etc.

url3 = "https://space-facts.com/mars/"

table = pd.read_html(url3)
df = table[0]
df.columns = ['Description','Value']
df.set_index('Description', inplace=True)
df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [171]:
#Use Pandas to convert the data to a HTML table string.
html_table =df.to_html()
#Get rid of new line symbols
new_table = html_table.replace('\n', '')
new_table

mars_dict['html_table'] = new_table

In [172]:
#Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the 
#hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

url4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url4)
browser.links.find_by_partial_text('Next')

hem = browser.html

soup_hem = bs(hem,'html.parser')

#Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary 
#for each hemisphere.

#Create empty dictionary of lists
hem_dict = []
data = soup_hem.find_all('div',class_="description")

for item in data:
    highres_url = f"https://astrogeology.usgs.gov{item.find('a')['href']}"
    highres = browser.visit(highres_url)
    highres = browser.html
    
    soup_highres = bs(highres,'html.parser')
    
    highres_img = soup_highres.find_all('div',class_='downloads')[0].find('a')['href']
    
    title = item.find('h3').text
    hem_dict.append({"title": title, "img_url": highres_img})
    
print(hem_dict)
mars_dict['hemisphere_image_urls']=hem_dict

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [173]:
#Print out mars_dict 
print(mars_dict)

{'news_title': "NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars", 'news_p': "In a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. NASA's Perseverance Rover Will Carry First Spacesuit Materials to MarsJuly 28, 2020NASA's Perseverance Rover Will Carry First Spacesuit Materials to MarsIn a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. ", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14924_ip.jpg', 'mars_weather': 'Mars Weather@MarsWxReport·5hInSight sol 594 (2020-07-28) low -91.8ºC (-133.2ºF) high -16.6ºC (2.2ºF)\nwinds from the WNW at 8.0 m/s (17.8 mph) gusting to 20.7 m/s (46.4 mph)\npressure at 7.90 hPa1118', 'html_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: ri